In [1]:
import sys
from sys import exit
import glob
import numpy as np
import pandas as pd
from os.path import join, isfile
from datetime import date, datetime
from unidecode import unidecode

from xlsxwriter.worksheet import (
    Worksheet, cell_number_tuple, cell_string_tuple)
    
from time import time, sleep
from contextlib import contextmanager
import functools
import threading


from my_funcs import ftime_elapsed, normalize_number, normalize_text, normalize_hash, trim_overspace, isvaliddate, normalize_cpf, normalize_date, date_hash, get_better, get_better_2, fit_cols, compare_dates, get_casos_confirmados, get_vacinados, get_tb_vacinados, get_srag

pd.set_option('display.max_columns', 0)
pd.options.display.float_format = lambda x: '{:,.0f}'.format(x).replace(',','.')

meses = ['Janeiro','Fevereiro','Março','Abril','Maio','Junho','Julho','Agosto','Setembro','Outubro','Novembro','Dezembro']
municipios = pd.read_csv('municipios.csv',converters={'ibge':str})
regionais = pd.read_csv('regionais.csv',converters={'ibge':str})

# faixa_etaria = [0,30,40,50,60,70,80,90,100]
# faixa_etaria_labels = ['err','0-29','30-39','40-49','50-59','60-69','70-79','80-89','90-99','>=100']
faixa_etaria = [60,65,70,75,80,85,90]
faixa_etaria_labels = ['0-59','60-64','65-69','70-74','75-79','80-84','85-89','>=90']
# faixa_etaria = [60,70,80,90]
# faixa_etaria_labels = ['0-59','60-69','70-79','80-89','>=90']

In [2]:
init_time = time()
vacinados = get_vacinados(load=False)
print(ftime_elapsed(init_time), 'vacinados shape:', vacinados.shape)
vacinados['mes_aplicacao'] = vacinados['data_aplicacao'].apply(lambda x: meses[x.month-1] if isinstance(x,date) else 'erro')
vacinados.groupby('mes_aplicacao')[['sexo']].count()

/home/vinnylg/projects/covid19datascience/vacinacao/my_funcs.py:230: FutureWarning: Comparison of NaT with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.
  if begin <= dt <= end:


00:08:16 vacinados shape: (2957518, 19)


,sexo
mes_aplicacao,
Abril,1303050
Fevereiro,254033
Janeiro,152407
Maio,303857
Março,944170


In [3]:
vacinados.columns

Index(['cns', 'paciente', 'cpf', 'sexo', 'nome_mae', 'data_nascimento', 'ibge',
       'vacina_categoria_nome', 'data_aplicacao', 'dose', 'fabricante',
       'vacina_grupoAtendimento_nome', 'vacina_nome', 'vacina_numDose',
       'sistema_origem', 'paciente_endereco_uf',
       'paciente_localNascimento_uf_codigo', 'hash_mae', 'hash_nasc',
       'mes_aplicacao'],
      dtype='object')

In [4]:
vacinados.groupby('dose')[['sexo']].count()

,sexo
dose,
1ª Dose,1970573
2ª Dose,986555
Única,368
Única,21


In [5]:
doses_header = ['cns','cpf','paciente','sexo','nome_mae','data_nascimento','ibge','data_aplicacao','fabricante','vacina_nome','hash_mae','hash_nasc'] #
primeira_dose = vacinados.loc[(vacinados['dose']=='1ª Dose')|(vacinados['dose']=='Única'),doses_header]
segunda_dose = vacinados.loc[vacinados['dose']=='2ª Dose',doses_header]

tb_vacinados = pd.merge(primeira_dose,segunda_dose, how='outer',on='cns',suffixes=['_1a_dose','_2a_dose'])

tb_vacinados['cpf'] = tb_vacinados.apply(lambda x: get_better(x['cpf_1a_dose'],x['cpf_2a_dose']), axis=1)
tb_vacinados['data_nascimento'] = tb_vacinados.apply(lambda x: get_better(x['data_nascimento_1a_dose'],x['data_nascimento_2a_dose'],datetime), axis=1)
tb_vacinados['nome_mae'] = tb_vacinados.apply(lambda x: get_better(x['nome_mae_1a_dose'],x['nome_mae_2a_dose']), axis=1)
tb_vacinados['paciente'] = tb_vacinados.apply(lambda x: get_better(x['paciente_1a_dose'],x['paciente_2a_dose']), axis=1)
tb_vacinados['fabricante'] = tb_vacinados.apply(lambda x: get_better(x['fabricante_1a_dose'],x['fabricante_2a_dose']), axis=1)
tb_vacinados['vacina_nome'] = tb_vacinados.apply(lambda x: get_better_2(x['vacina_nome_1a_dose'],x['vacina_nome_2a_dose']), axis=1)
tb_vacinados['hash_mae'] = tb_vacinados.apply(lambda x: get_better(x['hash_mae_1a_dose'],x['hash_mae_2a_dose']), axis=1)
tb_vacinados['hash_nasc'] = tb_vacinados.apply(lambda x: get_better(x['hash_nasc_1a_dose'],x['hash_nasc_2a_dose']), axis=1)
tb_vacinados['ibge'] = tb_vacinados.apply(lambda x: get_better(x['ibge_1a_dose'],x['ibge_2a_dose']), axis=1)
tb_vacinados['sexo'] = tb_vacinados.apply(lambda x: get_better(x['sexo_1a_dose'],x['sexo_2a_dose']), axis=1)

tb_vacinados = tb_vacinados.drop(columns=['cpf_1a_dose','cpf_2a_dose', 'data_nascimento_1a_dose', 'data_nascimento_2a_dose', 'paciente_1a_dose', 'paciente_2a_dose', 'fabricante_1a_dose', 'fabricante_2a_dose', 'vacina_nome_1a_dose', 'vacina_nome_2a_dose', 'nome_mae_1a_dose', 'nome_mae_2a_dose', 'hash_mae_1a_dose', 'hash_mae_2a_dose', 'hash_nasc_1a_dose', 'hash_nasc_2a_dose','ibge_1a_dose','ibge_2a_dose','sexo_2a_dose','sexo_1a_dose'])

tb_vacinados.loc[(tb_vacinados['data_aplicacao_1a_dose'].notnull()) & (tb_vacinados['data_aplicacao_2a_dose'].notnull()), 'dias_entre_doses'] = (
    tb_vacinados.loc[(tb_vacinados['data_aplicacao_1a_dose'].notnull()) & (tb_vacinados['data_aplicacao_2a_dose'].notnull()), 'data_aplicacao_2a_dose'] -
    tb_vacinados.loc[(tb_vacinados['data_aplicacao_1a_dose'].notnull()) & (tb_vacinados['data_aplicacao_2a_dose'].notnull()), 'data_aplicacao_1a_dose'])

tb_vacinados.loc[tb_vacinados['dias_entre_doses'].notnull(),'dias_entre_doses'] = tb_vacinados.loc[tb_vacinados['dias_entre_doses'].notnull(),'dias_entre_doses'].apply(lambda x: int(x.days))

tb_vacinados.loc[tb_vacinados['dias_entre_doses'].isnull(),'dias_entre_doses'] = 0

tb_vacinados['situacao'] = 'inconsistencia'

tb_vacinados.loc[(tb_vacinados['data_aplicacao_1a_dose'].notnull()) & (tb_vacinados['data_aplicacao_2a_dose'].isnull()), 'situacao'] = 'só primeira dose'

tb_vacinados.loc[(tb_vacinados['dias_entre_doses']>0), 'situacao'] = 'ambas doses aplicadas'

tb_vacinados['data_ultima_dose'] = tb_vacinados['data_aplicacao_1a_dose'].copy()
tb_vacinados.loc[tb_vacinados['data_aplicacao_2a_dose'].notna(),'data_ultima_dose'] = tb_vacinados.loc[tb_vacinados['data_aplicacao_2a_dose'].notna(),'data_aplicacao_2a_dose'].copy()

tb_vacinados['dias_apos_ultima_dose'] = tb_vacinados.apply(lambda x: (pd.to_datetime(date.today()) - x['data_ultima_dose']).days, axis=1)

vacinados['ibge'] = vacinados['ibge'].fillna('999999').apply(str)
regionais['ibge'] = regionais['ibge'].apply(str)

tb_vacinados = pd.merge(tb_vacinados,regionais[['ibge','nu_reg','nm_reg','nm_macro']],on='ibge',how='left')

tb_vacinados.loc[(tb_vacinados['data_nascimento'].notnull()) & (tb_vacinados['data_ultima_dose'].notnull()), 'idade'] = \
    tb_vacinados.loc[(tb_vacinados['data_nascimento'].notnull()) & (tb_vacinados['data_ultima_dose'].notnull())].apply(
            lambda row: row['data_ultima_dose'].year - row['data_nascimento'].year - (
                    (row['data_ultima_dose'].month, row['data_ultima_dose'].day) <
                    (row['data_nascimento'].month, row['data_nascimento'].day)
            ), axis=1
    )
tb_vacinados.loc[tb_vacinados['data_nascimento'].isna(), 'idade'] = -99
tb_vacinados['idade'] = tb_vacinados['idade'].apply(int)

tb_vacinados['faixa_etaria'] = [ faixa_etaria_labels[idx] for idx in np.digitize(tb_vacinados['idade'],faixa_etaria,right=False)]

print(len(tb_vacinados))
 
print(len(tb_vacinados.loc[tb_vacinados.duplicated("cns",keep="first")]))
tb_vacinados = tb_vacinados.drop_duplicates("cns", keep="first")

print(len(tb_vacinados.loc[tb_vacinados.duplicated("cpf",keep="first")]))
tb_vacinados = tb_vacinados.drop_duplicates("cpf", keep="first")

print(len(tb_vacinados.loc[tb_vacinados.duplicated("hash_mae",keep="first")]))
tb_vacinados = tb_vacinados.drop_duplicates("hash_mae", keep="first")

print(len(tb_vacinados.loc[tb_vacinados.duplicated("hash_nasc",keep="first")]))
tb_vacinados = tb_vacinados.drop_duplicates("hash_nasc", keep="first")

print(len(tb_vacinados))

tb_vacinados.groupby('faixa_etaria')[['cns']].count()

2075082
201056
197135
9096
321
1667474


,cns
faixa_etaria,
0-59,332147
60-64,350290
65-69,337630
70-74,265489
75-79,180792
80-84,113936
85-89,57595
>=90,29595


In [6]:
bins = [1,2,3,7,14,21,30,60,90]
bins_label = ['hoje','24 horas','48 horas', '72 horas', '7 dias', '14 dias', '21 dias', '30 dias', '60 dias', '90 dias']
tb_vacinados['periodo_ultima_dose'] = [ x for x in np.digitize(tb_vacinados['dias_apos_ultima_dose'],bins,right=False)]
dias_apos_ultima_dose = tb_vacinados.groupby(['situacao','periodo_ultima_dose'])[['cns']].count().unstack().fillna(0).astype(int).rename(columns={'cns':'qtde'}).droplevel(0,1)
dias_apos_ultima_dose['total'] = dias_apos_ultima_dose.sum(1)
dias_apos_ultima_dose = dias_apos_ultima_dose.append(pd.DataFrame(dias_apos_ultima_dose.sum().tolist(),index=dias_apos_ultima_dose.columns.tolist(),columns=['total']).T)
dias_apos_ultima_dose.columns = [ bins_label[bins] if isinstance(bins,int) else bins for bins in dias_apos_ultima_dose.columns ]
dias_apos_ultima_dose = dias_apos_ultima_dose.reset_index()
dias_apos_ultima_dose.index = [0,2,1,3]
dias_apos_ultima_dose = dias_apos_ultima_dose.sort_index().set_index('index')
dias_apos_ultima_dose.index.name = None
dias_apos_ultima_dose

,hoje,24 horas,48 horas,72 horas,7 dias,14 dias,21 dias,30 dias,60 dias,90 dias,total
ambas doses aplicadas,2,716,1902,9313,42589,158060,258049,164804,133720,9173,778328
só primeira dose,85,5830,11316,34375,164097,135151,148588,228116,100187,30085,857830
inconsistencia,0,131,121,409,2347,5590,11242,4935,6083,458,31316
total,87,6677,13339,44097,209033,298801,417879,397855,239990,39716,1667474


In [7]:
tb_vacinados['periodo_ultima_dose'] = tb_vacinados['periodo_ultima_dose'].apply(lambda x: bins_label[x]).copy()
tb_vacinados = tb_vacinados[['ibge','nu_reg','nm_reg','nm_macro','cns', 'cpf', 'paciente', 'sexo', 'data_nascimento', 'idade', 'faixa_etaria', 'nome_mae', 'vacina_nome', 'fabricante', 'data_aplicacao_1a_dose', 'data_aplicacao_2a_dose', 'dias_entre_doses', 'situacao', 'dias_apos_ultima_dose', 'periodo_ultima_dose', 'data_ultima_dose', 'hash_mae', 'hash_nasc']]

In [8]:
tb_vacinados = tb_vacinados.reset_index()
tb_vacinados.to_feather('tb_vacinados.feather')

In [2]:
tb_vacinados = pd.read_feather('tb_vacinados.feather')
tb_vacinados = tb_vacinados.set_index('index')

In [3]:
casos_confirmados = get_casos_confirmados(load=True)
casos_confirmados['faixa_etaria'] = [ faixa_etaria_labels[idx] for idx in np.digitize(casos_confirmados['idade'],faixa_etaria,right=False)]
casos_confirmados.groupby('evolucao')[['id']].count()

,id
evolucao,
ATIVO,308409
CURA,628491
OBITO,23423


In [4]:
print('casos_confirmados shape:', casos_confirmados.shape)
casos_confirmados.groupby('faixa_etaria')[['id']].count()

casos_confirmados shape: (960323, 34)


,id
faixa_etaria,
0-59,824498
60-64,46829
65-69,34251
70-74,23169
75-79,14912
80-84,9102
85-89,4921
>=90,2641


In [5]:
casos_confirmados['comorbidade'] = casos_confirmados.apply(lambda x: 1 if any([
    x['doenca_cardiovascular'] == 1,
    x['hipertensao'] == 1,
    x['diabetes'] == 1,
    x['doenca_hepatica'] == 1,
    x['doenca_neurologica'] == 1,
    x['sindrome_down'] == 1,
    x['imunodeficiencia'] == 1,
    x['infeccao_hiv'] == 1,
    x['doenca_renal'] == 1,
    x['doenca_pulmonar'] == 1,
    x['neoplasia'] == 1,
    x['puerperio'] == 1,
    x['obesidade'] == 1,
    x['tabagismo'] == 1
]) else 0, axis=1)
casos_confirmados.groupby('comorbidade')[['id']].count()

,id
comorbidade,
0,809524
1,150799


In [6]:
obitos_confirmados = casos_confirmados.loc[(casos_confirmados['evolucao']=='OBITO')]
print('obitos_confirmados shape:', obitos_confirmados.shape)
obitos_confirmados.groupby('faixa_etaria')[['id']].count()

obitos_confirmados shape: (23423, 35)


,id
faixa_etaria,
0-59,6647
60-64,2747
65-69,3228
70-74,3175
75-79,2927
80-84,2233
85-89,1518
>=90,948


In [7]:
casos_vacinados = casos_confirmados.loc[
    (casos_confirmados['cns'].notnull() & casos_confirmados['cns'].isin(tb_vacinados['cns'])) |
    (casos_confirmados['cpf'].notnull() & casos_confirmados['cpf'].isin(tb_vacinados['cpf'])) |
    (casos_confirmados['hash_mae'].notnull() & casos_confirmados['hash_mae'].isin(tb_vacinados['hash_mae'])) |
    (casos_confirmados['hash_nasc'].notnull() & casos_confirmados['hash_nasc'].isin(tb_vacinados['hash_nasc']))
]

print(len(casos_vacinados))

133547


In [8]:
tb_vacinados_cols = ['situacao','data_ultima_dose','vacina_nome','fabricante','data_aplicacao_1a_dose','data_aplicacao_2a_dose']

casos_vacinados = pd.merge(casos_confirmados.loc[casos_confirmados['cns'].notna()],tb_vacinados.loc[tb_vacinados['cns'].notna(),['cns']+tb_vacinados_cols], on='cns', how='inner')
tb_vacinados = tb_vacinados.drop(index=tb_vacinados.loc[tb_vacinados['cns'].isin(casos_vacinados['cns'])].index)
print(len(casos_vacinados))
print(len(tb_vacinados))

casos_vacinados = casos_vacinados.append(pd.merge(casos_confirmados.loc[casos_confirmados['cpf'].notna()],tb_vacinados.loc[tb_vacinados['cpf'].notna(),['cpf']+tb_vacinados_cols], on='cpf', how='inner'), ignore_index=True)
tb_vacinados = tb_vacinados.drop(index=tb_vacinados.loc[tb_vacinados['cpf'].isin(casos_vacinados['cpf'])].index)
print(len(casos_vacinados))
print(len(tb_vacinados))

casos_vacinados = casos_vacinados.append(pd.merge(casos_confirmados.loc[casos_confirmados['hash_mae'].notna()],tb_vacinados.loc[tb_vacinados['hash_mae'].notna(),['hash_mae']+tb_vacinados_cols], on='hash_mae', how='inner'), ignore_index=True)
tb_vacinados = tb_vacinados.drop(index=tb_vacinados.loc[tb_vacinados['hash_mae'].isin(casos_vacinados['hash_mae'])].index)
print(len(casos_vacinados))
print(len(tb_vacinados))

casos_vacinados = casos_vacinados.append(pd.merge(casos_confirmados.loc[casos_confirmados['hash_nasc'].notna()],tb_vacinados.loc[tb_vacinados['hash_nasc'].notna(),['hash_nasc']+tb_vacinados_cols], on='hash_nasc', how='inner'), ignore_index=True)
tb_vacinados = tb_vacinados.drop(index=tb_vacinados.loc[tb_vacinados['hash_nasc'].isin(casos_vacinados['hash_nasc'])].index)
print(len(casos_vacinados))
print(len(tb_vacinados))

print(len(casos_vacinados.loc[casos_vacinados.duplicated('id',keep=False)]))
casos_vacinados = casos_vacinados.drop_duplicates('id',keep='first')
len(casos_vacinados)

111454
1556030
125402
1542089
129214
1538399
131508
1536178
26


131495

In [9]:
casos_vacinados['diferenca_vacina_diag'] = casos_vacinados['data_diagnostico'] - casos_vacinados['data_ultima_dose']
casos_vacinados['diferenca_vacina_diag'] = casos_vacinados['diferenca_vacina_diag'].apply(lambda x: x.days)
casos_vacinados['situacao'].unique()
# casos_vacinados.groupby('vacina_nome')[['id']].count()

array(['ambas doses aplicadas', 'só primeira dose', 'inconsistencia'],
      dtype=object)

In [10]:
casos = pd.merge(casos_confirmados,casos_vacinados[['id','diferenca_vacina_diag']+tb_vacinados_cols],how='left',on='id')
casos['vacinado'] = 0 # sem vacina
casos.loc[(casos['diferenca_vacina_diag']>=0)&(casos['diferenca_vacina_diag']<=14),'vacinado'] = 1 #com vacina
casos.loc[(casos['situacao']=='ambas doses aplicadas')&(casos['diferenca_vacina_diag']>14),'vacinado'] = 2 #imunizado
casos['ano_caso'] = casos['data_diagnostico'].apply(lambda x: x.year if isinstance(x,date) else 'erro')
casos['mes_caso'] = casos['data_diagnostico'].apply(lambda x: x.month if isinstance(x,date) else 'erro')
grupo = casos.groupby(['ano_caso','mes_caso','faixa_etaria','vacinado'])[['id']].count().unstack(-2)
grupo = grupo.droplevel(0,1)
grupo = grupo.fillna(0).astype(float)
grupo.to_excel('casos_vacinados.xlsx')
grupo

faixa_etaria                  0-59  60-64  65-69  ...  80-84  85-89  >=90
ano_caso mes_caso vacinado                        ...                    
2.020    3        0            295     21      8  ...      5      2     2
         4        0          1.301     61     51  ...     23     12     5
         5        0          4.535    225    168  ...     57     39    10
         6        0         27.776  1.312    953  ...    292    169    96
         7        0         50.817  2.607  1.887  ...    558    350   217
         8        0         48.245  2.569  1.817  ...    540    331   201
         9        0         39.256  2.090  1.524  ...    415    248   151
         10       0         30.131  1.581  1.175  ...    350    190    97
         11       0         92.734  4.783  3.383  ...    830    419   238
         12       0        106.665  6.116  4.596  ...  1.223    679   338
2.021    1        0         93.931  5.501  4.197  ...  1.173    593   354
                  1             56      1      4  ...      1      2     4
         2        0         89.012  5.485  3.955  ...  1.049    540   250
                  1            662     20     10  ...      9     37    42
                  2              5      0      0  ...      0      0     0
         3        0        129.074  8.002  5.992  ...  1.213    597   258
                  1            600     20     28  ...    366    202   105
                  2            942     25     28  ...     18     26    31
         4        0         73.033  4.214  2.537  ...    499    246    73
                  1            302    254    809  ...    126     21     5
                  2            895     42     22  ...    115     97    91
         5        0         33.606  1.296    773  ...    151     77    30
                  1            169    592    290  ...      7      3     0
                  2            456     12     44  ...     82     41    34

[24 rows x 8 columns]

In [11]:
obitos_vacinados = casos_vacinados.loc[casos_vacinados['evolucao']=='OBITO'].copy()
obitos_vacinados['diferenca_vacina_obito'] = obitos_vacinados['data_cura_obito'] - obitos_vacinados['data_ultima_dose']
obitos_vacinados['diferenca_vacina_obito'] = obitos_vacinados['diferenca_vacina_obito'].apply(lambda x: x.days)

In [12]:
obitos = pd.merge(obitos_confirmados,obitos_vacinados[['id','diferenca_vacina_diag']+tb_vacinados_cols],how='left',on='id')
obitos['vacinado'] = 0 # sem vacina
obitos.loc[(obitos['diferenca_vacina_diag']>=0)&(obitos['diferenca_vacina_diag']<=14),'vacinado'] = 1 #com vacina
obitos.loc[(obitos['situacao']=='ambas doses aplicadas')&(obitos['diferenca_vacina_diag']>14),'vacinado'] = 2 #imunizado

obitos['ano_obito'] = obitos['data_cura_obito'].apply(lambda x: x.year if isinstance(x,date) else 'erro')
obitos['mes_obito'] = obitos['data_cura_obito'].apply(lambda x: x.month if isinstance(x,date) else 'erro')
grupo = obitos.groupby(['ano_obito','mes_obito','faixa_etaria','vacinado'])[['id']].count().unstack(-2)
grupo = grupo.droplevel(0,1)

grupo = grupo.fillna(0).astype(float)
    
grupo.to_excel('obitos_vacinados.xlsx')
grupo

faixa_etaria                  0-59  60-64  65-69  ...  80-84  85-89  >=90
ano_obito mes_obito vacinado                      ...                    
2.020     3         0            2      1      1  ...      1      0     0
          4         0           19      1      5  ...      7      6     5
          5         0           22     12     12  ...      7      9     2
          6         0          127     70     59  ...     60     62    27
          7         0          353    118    169  ...    149    107    80
          8         0          331    167    174  ...    146    120    80
          9         0          256    111    133  ...    111     88    80
          10        0          164     74    111  ...    101     60    42
          11        0          219    111    154  ...    126     94    59
          12        0          492    250    307  ...    253    177    99
2.021     1         0          408    217    264  ...    227    147   108
          2         0          476    219    251  ...    200    130    71
                    1            1      1      2  ...      1      6    17
          3         0        1.942    718    827  ...    437    259   140
                    1           11      0      2  ...    104     80    54
                    2            1      0      1  ...      2      3     1
          4         0        1.372    519    544  ...    179     94    30
                    1            3      1     52  ...     57     24     9
                    2            4      1      1  ...     13     19    18
          5         0          430    138    121  ...     33     19    12
                    1            1     15     33  ...      6      1     0
                    2            2      0      1  ...      8     12    12

[22 rows x 8 columns]

In [44]:
obitos = pd.merge(obitos_confirmados,obitos_vacinados[['id','diferenca_vacina_diag']+tb_vacinados_cols],how='left',on='id')
obitos['vacinado'] = 0 # sem vacina
obitos.loc[(obitos['diferenca_vacina_diag']>=0)&(obitos['diferenca_vacina_diag']<=14),'vacinado'] = 1 #com vacina
obitos.loc[(obitos['situacao']=='ambas doses aplicadas')&(obitos['diferenca_vacina_diag']>14),'vacinado'] = 2 #imunizado

obitos = obitos.loc[(obitos['data_cura_obito'] > '2020-06-30') & (obitos['data_cura_obito'] < '2021-05-01' ) ]

grupo = obitos.groupby(['faixa_etaria','vacinado','comorbidade'])[['id']].count().unstack(-2).unstack(-1)

grupo = grupo.fillna(0).astype(int)
    
grupo.to('comorbidade_raw.xlsx')
grupo

id                       
vacinado         0         1        2    
comorbidade      0     1   0    1   0   1
faixa_etaria                             
0-59          3127  2886  10    5   0   5
60-64         1150  1354   2    0   0   1
65-69         1222  1712  28   28   1   1
70-74         1067  1782  38   66   1   3
75-79          981  1617  58  104   4   2
80-84          742  1187  67   95   4  11
85-89          484   792  46   64  12  10
>=90           336   453  33   47   8  11

In [43]:
tb_vacinados = pd.read_feather('tb_vacinados.feather')
tb_vacinados = tb_vacinados.set_index('index')

srag = get_srag(load=True)
srag = srag.loc[(srag['classi_fin']=='5')&(srag['sg_uf']=='PR')]

In [15]:
srag['nu_idade_n'] = srag['nu_idade_n'].apply(int)
srag['faixa_etaria'] = [ faixa_etaria_labels[idx] for idx in np.digitize(srag['nu_idade_n'],faixa_etaria,right=False)]
obitos_srag = srag.loc[srag['evolucao']=='2']

In [16]:
srag_vacinados = srag.loc[
    (srag['nu_cns'].notnull() & srag['nu_cns'].isin(tb_vacinados['cns'])) |
    (srag['nu_cpf'].notnull() & srag['nu_cpf'].isin(tb_vacinados['cpf'])) |
    (srag['hash_mae'].notnull() & srag['hash_mae'].isin(tb_vacinados['hash_mae'])) |
    (srag['hash_nasc'].notnull() & srag['hash_nasc'].isin(tb_vacinados['hash_nasc']))
]

print(len(srag_vacinados))
tb_vacinados.columns

16573


Index(['ibge', 'nu_reg', 'nm_reg', 'nm_macro', 'cns', 'cpf', 'paciente',
       'sexo', 'data_nascimento', 'idade', 'faixa_etaria', 'nome_mae',
       'vacina_nome', 'fabricante', 'data_aplicacao_1a_dose',
       'data_aplicacao_2a_dose', 'dias_entre_doses', 'situacao',
       'dias_apos_ultima_dose', 'periodo_ultima_dose', 'data_ultima_dose',
       'hash_mae', 'hash_nasc'],
      dtype='object')

In [17]:
tb_vacinados_cols = ['paciente','nome_mae','data_nascimento','situacao','data_ultima_dose','vacina_nome','fabricante','data_aplicacao_1a_dose','data_aplicacao_2a_dose']
# tb_vacinados[tb_vacinados_cols]
srag_vacinados = pd.merge(srag.loc[srag['nu_cns'].notna()].rename(columns={'nu_cns':'cns'}),tb_vacinados.loc[tb_vacinados['cns'].notna(),['cns']+tb_vacinados_cols], on='cns', how='inner')
tb_vacinados = tb_vacinados.drop(index=tb_vacinados.loc[tb_vacinados['cns'].isin(srag_vacinados['cns'])].index)
print(len(srag_vacinados))
print(len(tb_vacinados))

srag_vacinados = srag_vacinados.append(pd.merge(srag.loc[srag['nu_cpf'].notna()].rename(columns={'nu_cpf':'cpf'}),tb_vacinados.loc[tb_vacinados['cpf'].notna(),['cpf']+tb_vacinados_cols], on='cpf', how='inner'), ignore_index=True)
tb_vacinados = tb_vacinados.drop(index=tb_vacinados.loc[tb_vacinados['cpf'].isin(srag_vacinados['cpf'])].index)
print(len(srag_vacinados))
print(len(tb_vacinados))

srag_vacinados = srag_vacinados.append(pd.merge(srag.loc[srag['hash_mae'].notna()],tb_vacinados.loc[tb_vacinados['hash_mae'].notna(),['hash_mae']+tb_vacinados_cols], on='hash_mae', how='inner'), ignore_index=True)
tb_vacinados = tb_vacinados.drop(index=tb_vacinados.loc[tb_vacinados['hash_mae'].isin(srag_vacinados['hash_mae'])].index)
print(len(srag_vacinados))
print(len(tb_vacinados))

srag_vacinados = srag_vacinados.append(pd.merge(srag.loc[srag['hash_nasc'].notna()],tb_vacinados.loc[tb_vacinados['hash_nasc'].notna(),['hash_nasc']+tb_vacinados_cols], on='hash_nasc', how='inner'), ignore_index=True)
tb_vacinados = tb_vacinados.drop(index=tb_vacinados.loc[tb_vacinados['hash_nasc'].isin(srag_vacinados['hash_nasc'])].index)
print(len(srag_vacinados))
print(len(tb_vacinados))

print(len(srag_vacinados.loc[srag_vacinados.duplicated('nu_notific',keep=False)]))
srag_vacinados = srag_vacinados.drop_duplicates('nu_notific',keep='first')
len(srag_vacinados)

552
1666940
14562
1653466
15829
1652210
16402
1651640
4


16400

In [18]:
srag_vacinados.loc[(
#     (srag_vacinados['nu_cns'] != srag_vacinados['cns']) &
#     (srag_vacinados['nu_cpf'] != srag_vacinados['cpf']) &
    (srag_vacinados['nm_pacient'] != srag_vacinados['paciente']) 
#     (srag_vacinados['dt_nasc'] != srag_vacinados['data_nascimento']) &
#     (srag_vacinados['nm_mae_pac'] != srag_vacinados['nome_mae'])
),['nu_cns','cns','nu_cpf','cpf','nm_pacient','paciente','dt_nasc','data_nascimento','nm_mae_pac','nome_mae']]

,nu_cns,cns,nu_cpf,cpf,nm_pacient,paciente,dt_nasc,data_nascimento,nm_mae_pac,nome_mae
114,NaN,705008000730753,19314477991,NaN,JURANDYR FIORI,JURANDIR FIORI,1936-11-13,1936-11-13,JOSEFINA FIORI,JOSEFINA FIORI
197,NaN,701103875637080,58868380900,NaN,REGINA DIAS DOS SANTOS,REGINA DIAS SANTOS,1937-01-15,1934-01-15,NaN,FRANCISCA DA CONCEICAO
250,NaN,708404719492769,44944691904,NaN,BENEDITA DE SOUZA SILVA,JORGE SIDNEI PIROLO,1943-12-28,1957-02-02,BRAZILINA GOMES DE SOUZA,ERCINA CONCEICAO DE JESUS PIROLO
258,NaN,706807151866530,02340399904,NaN,JOSE MARIA DE CASTRO,MARIA GRANATYR DOFAT,1933-05-12,1927-08-19,AMELIA MARQUES DE CASTRO,ROSALIA PISKLEWICZ GRANATYR
380,NaN,706709509155214,62644254949,NaN,MARIA ELIZETE MACHADO,MARIA ELIZETE MACHADO NETO,1960-10-17,1960-10-17,MARIA TEREZA DEKI MACHADO,MARIA TEREZA DEKI MACHADO
...,...,...,...,...,...,...,...,...,...,...
15816,NaN,NaN,NaN,NaN,MARLENE SANT'ANA PEREIRA,MARLENE SANT ANA PEREIRA,1950-10-30,1950-10-30,MARIA GERALDINA DA SILVA,MARIA GERALDINA DA SILVA
16101,NaN,NaN,NaN,NaN,DANIEL INACIO DA SILVA,DANIEL INACIO DA SILVA,1959-07-28,1959-07-28,RAIMUNDA MATIAS DA SILVA,RAIMUNDA DA SILVA
16131,NaN,NaN,NaN,NaN,JOSEFINA IZELMA BAGATINPASQUETO,JOSEFINA IZELMA BAGATIN PASQUETO,1939-03-10,1939-03-10,LUCIA,LUCIA SASDELI BAGATIN
16204,NaN,NaN,NaN,NaN,CIRCEA DALVA PERROUD SILVA,CIRCEA D ALVA PERROUD SILVA,1940-01-06,1940-01-06,RIZIA MARTINS DE MELLO,ORIZIA MARTINS DE MELO


In [19]:
srag_vacinados.groupby('vacina_nome')[['nu_notific']].count()

,nu_notific
vacina_nome,
Covid-19-AstraZeneca,161
Covid-19-Coronavac-Sinovac/Butantan,9658
Vacina Covid-19 - Covishield,6487
Vacina covid-19 - BNT162b2 - BioNTech/Fosun Pharma/Pfizer,94


In [20]:
srag_vacinados.groupby('evolucao')[['nu_notific']].count()

,nu_notific
evolucao,
1,13493
2,1268
3,17
9,246


In [21]:
srag_vacinados.loc[srag_vacinados['evolucao']=='2']

,nu_notific,dt_notific,nu_cpf,nm_pacient,cs_sexo,dt_nasc,nu_idade_n,nm_mae_pac,sg_uf,co_mun_res,fator_risc,hospital,dt_interna,uti,dt_coleta,pcr_resul,dt_pcr,classi_fin,evolucao,dt_evoluca,pcr_sars2,an_sars2,cns,hash_mae,hash_nasc,hash_resid,faixa_etaria,paciente,nome_mae,data_nascimento,situacao,data_ultima_dose,vacina_nome,fabricante,data_aplicacao_1a_dose,data_aplicacao_2a_dose,cpf,nu_cns
94,316159226482,16/03/2021,32086393920,MOACIR RODRIGUES,M,1939-01-05,82,FERMINA RODRIGUES,PR,410690,1,1,2021-03-15,2,19/03/2021,1,2021-03-23,5,2,2021-04-06,1,NaN,703405035702400,MOACIRRODRIGUESFERMINARODRIGUES,MOACIRRODRIGUES05011939,MOACIRRODRIGUES410690,80-84,MOACIR RODRIGUES,FERMINA RODRIGUES,1939-01-05,só primeira dose,2021-03-04,Vacina Covid-19 - Covishield,SERUM INSTITUTE OF INDIA LTD.,2021-03-04,NaT,NaN,NaN
108,316161773800,18/03/2021,45137293920,MIRO ALBINO STEYER,M,1944-08-07,76,LUCINDA STEYER,PR,412480,1,1,2021-03-18,2,18/03/2021,1,2021-03-22,5,2,2021-03-28,NaN,NaN,700402503640150,MIROALBINOSTEYERLUCINDASTEYER,MIROALBINOSTEYER07081944,MIROALBINOSTEYER412480,75-79,MIRO ALBINO STEYER,LUCINDA STEYER,1944-08-07,só primeira dose,2021-03-12,Covid-19-Coronavac-Sinovac/Butantan,FUNDACAO BUTANTAN,2021-03-12,NaT,NaN,NaN
114,316164319636,20/03/2021,19314477991,JURANDYR FIORI,M,1936-11-13,84,JOSEFINA FIORI,PR,410650,1,1,2021-03-19,2,20/03/2021,1,2021-03-23,5,2,2021-03-23,NaN,NaN,705008000730753,JURANDYRFIORIJOSEFINAFIORI,JURANDYRFIORI13111936,JURANDYRFIORI410650,80-84,JURANDIR FIORI,JOSEFINA FIORI,1936-11-13,só primeira dose,2021-03-05,Covid-19-Coronavac-Sinovac/Butantan,FUNDACAO BUTANTAN,2021-03-05,NaT,NaN,NaN
121,316165925401,24/03/2021,53602609987,JOANA ALVES DAS ALMAS,F,1932-06-22,88,CLAUDINA GALVAO DAS ALMAS,PR,410400,1,1,2021-03-17,2,15/03/2021,1,2021-03-17,5,2,2021-03-20,1,NaN,706207085390966,JOANAALVESDASALMASCLAUDINAGALVAODASALMAS,JOANAALVESDASALMAS22061932,JOANAALVESDASALMAS410400,85-89,JOANA ALVES DAS ALMAS,CLAUDINA GALVAO DAS ALMAS,1932-06-22,só primeira dose,2021-03-01,Covid-19-Coronavac-Sinovac/Butantan,FUNDACAO BUTANTAN,2021-03-01,NaT,NaN,NaN
124,316166047613,24/03/2021,73115720963,TERESINHA LUCI ROVER BROCHARD,F,1941-02-23,80,PEDRINA RIBEIRO ROVER,PR,410690,1,1,2021-03-24,1,24/03/2021,1,2021-03-25,5,2,2021-05-01,1,NaN,705008841977251,TERESINHALUCIROVERBROCHARDPEDRINARIBEIROROVER,TERESINHALUCIROVERBROCHARD23021941,TERESINHALUCIROVERBROCHARD410690,80-84,TERESINHA LUCI ROVER BROCHARD,PEDRINA RIBEIRO ROVER,1941-02-23,só primeira dose,2021-03-09,Vacina Covid-19 - Covishield,SERUM INSTITUTE OF INDIA LTD.,2021-03-09,NaT,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16373,316182562428,12/04/2021,NaN,NAIR TEREZA CHELA,F,1953-01-30,68,NaN,PR,410690,1,1,2021-04-11,2,08/04/2021,1,2021-04-10,5,2,2021-04-25,1,NaN,NaN,NaN,NAIRTEREZACHELA30011953,NAIRTEREZACHELA410690,65-69,NAIR TEREZA CHELA,JULIETA ADELIA BUGALSKI,1953-01-30,só primeira dose,2021-03-30,Covid-19-Coronavac-Sinovac/Butantan,FUNDACAO BUTANTAN,2021-03-30,NaT,NaN,NaN
16378,316188382385,19/04/2021,NaN,LORITA DE SOUZA DA TRINDADE,F,1948-10-07,72,ROSILBA PEREIRA DE SOUZA,PR,410690,1,1,2021-04-19,1,15/04/2021,1,2021-04-16,5,2,2021-04-22,1,NaN,NaN,LORITADESOUZADATRINDADEROSILBAPEREIRADESOUZA,LORITADESOUZADATRINDADE07101948,LORITADESOUZADATRINDADE410690,70-74,LORITA DE SOUZA DA TRINDADE,ROZELBA PEREIRA DE SOUZA,1948-10-07,só primeira dose,2021-03-24,Covid-19-Coronavac-Sinovac/Butantan,FUNDACAO BUTANTAN,2021-03-24,NaT,NaN,None
16382,316189258509,20/04/2021,NaN,ALAYDE DE OLIVEIRA COELHO,F,1937-03-20,84,MARIA MATILDE DE OLIVEIRA,PR,411915,2,1,2021-04-20,1,14/04/2021,1,2021-04-16,5,2,2021-04-21,1,NaN,NaN,ALAYDEDEOLIVEIRACOELHOMARIAMATILDEDEOLIVEIRA,ALAYDEDEOLIVEIRACOELHO20031937,ALAYDEDEOLIVEIRACOELHO411915,80-84,ALAYDE DE OLIVEIRA COELHO,MARIA MATILDE DE MELO,1937-03-20,ambas doses aplicadas,2021-03-30,Covid-19-Coronavac-Sinovac/Butantan,FUNDACAO BUTANTAN,2021-03-08,2021-0

In [37]:
srag_vacinados['diferenca_vacina_diag'] = srag_vacinados['dt_pcr'] - srag_vacinados['data_ultima_dose']
srag_vacinados['diferenca_vacina_diag'] = srag_vacinados['diferenca_vacina_diag'].apply(lambda x: x.days)

casos = pd.merge(srag,srag_vacinados[['nu_notific','diferenca_vacina_diag']+tb_vacinados_cols],how='left',on='nu_notific')
casos['vacinado'] = 0 # sem vacina
casos.loc[(casos['diferenca_vacina_diag']>=0)&(casos['diferenca_vacina_diag']<=14),'vacinado'] = 1 #com vacina
casos.loc[(casos['situacao']=='ambas doses aplicadas')&(casos['diferenca_vacina_diag']>14),'vacinado'] = 2 #imunizado


casos['ano_caso'] = casos['dt_pcr'].apply(lambda x: x.year if isinstance(x,date) else 'erro')
casos['mes_caso'] = casos['dt_pcr'].apply(lambda x: x.month if isinstance(x,date) else 'erro')
grupo = casos.groupby(['ano_caso','mes_caso','faixa_etaria','vacinado'])[['nu_notific']].count().unstack(-2)
grupo = grupo.droplevel(0,1)

grupo = grupo.fillna(0).astype(float)

grupo.to_excel('casos_srag_vacinados.xlsx')
grupo

faixa_etaria                0-59  60-64  65-69  ...  80-84  85-89  >=90
ano_caso mes_caso vacinado                      ...                    
2.020    3        0           43     13      9  ...      0      0     0
         4        0          222     43     25  ...     32     18     9
         5        0          452     73     58  ...     30     23     9
         6        0        1.434    275    247  ...    134    107    44
         7        0        2.742    622    573  ...    302    194   123
         8        0        2.277    493    490  ...    265    175   126
         9        0        1.724    424    382  ...    229    161   110
         10       0        1.259    292    307  ...    175    118    61
         11       0        2.501    562    603  ...    302    155   100
         12       0        3.018    776    811  ...    493    287   157
2.021    1        0        2.339    656    691  ...    364    223   134
                  1            4      2      0  ...      0      2     1
         2        0        3.575    783    732  ...    367    208   109
                  1           11      2      2  ...      2     19    16
         3        0        6.372  1.412  1.402  ...    483    245   108
                  1           18      0      4  ...    109     62    32
                  2           13      2      0  ...      4      8    10
         4        0        3.581    728    609  ...    158     61    24
                  1           13     12    135  ...     32      9     0
                  2           14      2      3  ...     25     36    28
         5        0          367     55     43  ...     23      5     3
                  1            1     15     14  ...      0      0     0
                  2            4      0      1  ...      6      6     5

[23 rows x 8 columns]

In [36]:
obitos_srag_vacinados = srag_vacinados.loc[srag_vacinados['evolucao']=='2'].copy()
obitos_srag_vacinados['diferenca_vacina_obito'] = obitos_srag_vacinados['dt_evoluca'] - obitos_srag_vacinados['data_ultima_dose']
obitos_srag_vacinados['diferenca_vacina_obito'] = obitos_srag_vacinados['diferenca_vacina_obito'].apply(lambda x: x.days)

obitos = pd.merge(obitos_srag,obitos_srag_vacinados[['nu_notific','diferenca_vacina_diag']+tb_vacinados_cols],how='left',on='nu_notific')
obitos['vacinado'] = 0 # sem vacina
obitos.loc[(obitos['diferenca_vacina_diag']>=0)&(obitos['diferenca_vacina_diag']<=14),'vacinado'] = 1 #com vacina
obitos.loc[(obitos['situacao']=='ambas doses aplicadas')&(obitos['diferenca_vacina_diag']>14),'vacinado'] = 2 #imunizado

obitos['ano_obito'] = obitos['dt_evoluca'].apply(lambda x: x.year if isinstance(x,date) else 'erro')
obitos['mes_obito'] = obitos['dt_evoluca'].apply(lambda x: x.month if isinstance(x,date) else 'erro')
grupo = obitos.groupby(['ano_obito','mes_obito','faixa_etaria','vacinado'])[['nu_notific']].count().unstack(-2)
grupo = grupo.droplevel(0,1)

grupo = grupo.fillna(0).astype(float)
    
grupo.to_excel('obitos_srag_vacinados.xlsx')
grupo

faixa_etaria                  0-59  60-64  65-69  ...  80-84  85-89  >=90
ano_obito mes_obito vacinado                      ...                    
2.020     3         0            2      0      1  ...      0      0     0
          4         0           31      7      5  ...     13      8     7
          5         0           29     16     12  ...     11     14     4
          6         0          137     70     61  ...     68     63    25
          7         0          343    126    157  ...    144    100    74
          8         0          332    172    183  ...    137    112    77
          9         0          261    110    146  ...    121     88    82
          10        0          153     75    117  ...     96     55    39
          11        0          188     98    131  ...    110     80    49
          12        0          450    231    290  ...    232    158    91
2.021     1         0          380    193    248  ...    202    140    89
                    1            0      0      0  ...      0      0     0
          2         0          416    205    237  ...    186    118    66
                    1            0      1      1  ...      1      5     8
          3         0        1.590    549    693  ...    352    190   107
                    1            7      0      1  ...     63     43    22
                    2            0      0      0  ...      2      2     2
          4         0        1.227    467    492  ...    171     83    29
                    1            3      3     38  ...     29     15     4
                    2            3      1      1  ...     11     16    15
          5         0          262     90     81  ...     23     10     6
                    1            2      3     16  ...      0      0     0
                    2            2      0      1  ...      5      4     5

[23 rows x 8 columns]

In [24]:
obitos_srag_vacinados.to_excel('obitos_srag_vacinados_data.xlsx')

In [25]:
len(obitos)

20632

In [33]:
casos.loc[casos['vacina_nome'].isna(),'vacina_nome'] = 'SEM VACINA'
casos.groupby(['vacinado','vacina_nome'])[['nu_notific']].count().unstack(0).fillna(0)

nu_notific        
vacinado                                                    0   1   2
vacina_nome                                                          
Covid-19-AstraZeneca                                      154   7   0
Covid-19-Coronavac-Sinovac/Butantan                     8.797 660 201
SEM VACINA                                             54.389   0   0
Vacina Covid-19 - Covishield                            6.294 193   0
Vacina covid-19 - BNT162b2 - BioNTech/Fosun Pha...         94   0   0

In [32]:
obitos.groupby(['ano_obito'])[['nu_notific']].count().unstack(0).fillna(0)

            ano_obito
nu_notific  2.020         8265
            2.021        12348
dtype: int64